In [1]:
import random
import sys
import numpy as np
import math
import pygame
def postavi_parametre():
    global ljubicasta,tamno_plava, Red,Stupac,duzina_niza,roza,UmjetnaInt_zeton, Prazno, covjek_zeton,narancasta, tamno_zelena, bijela
    ljubicasta =(128, 0, 1285)
    tamno_plava= (0, 0, 139)
    Red=6
    Stupac =7
    duzina_niza = 4
    roza =(255, 192, 203)
    UmjetnaInt_zeton= 2
    Prazno =0
    covjek_zeton =1
    narancasta=(255, 165, 0)
    tamno_zelena=(0, 128, 0)
    bijela=(255, 255, 255)

postavi_parametre()


def procjeni(rupa, zeton): 
    zgoditak= 0
    opp_zeton= UmjetnaInt_zeton if zeton == covjek_zeton else covjek_zeton
    if rupa.count(zeton) == 4:
        zgoditak += 200
    elif rupa.count(zeton)==3 and rupa.count(Prazno)== 1:
        zgoditak += 10
    elif rupa.count(zeton)== 2 and rupa.count(Prazno) ==2:
        zgoditak += 4
    if rupa.count(opp_zeton)==3 and rupa.count(Prazno) ==1:
        zgoditak -= 8

    return zgoditak


def procjeni_poziciju(ploca, zeton):
    

    ## zgoditak center stupacumn
    srednje_brojanje= np.count_nonzero(ploca[:, Stupac // 2]== zeton)
    zgoditak=0
    zgoditak += srednje_brojanje * 3

    ##Horizontalo
    r = 0
    while r<Red:
        red_polja= [int(i) for i in list(ploca[r, :])]
        c= 0
        while c< Stupac - 3:
            rupa= red_polja[c:c + duzina_niza]
            zgoditak += procjeni(rupa, zeton)
            c += 1
        r += 1

    ##Vertikalno
    c= 0
    while c <Stupac:
        stupac_polja =[int(i) for i in list(ploca[:, c])]
        r =0
        while r < Red -3:
            rupa = stupac_polja[r:r + duzina_niza]
            zgoditak +=procjeni(rupa, zeton)
            r +=1
        c +=1

    ##dijagonalu prema gore
    r =0
    while r<Red -3:
        c= 0
        while c < Stupac - 3:
            rupa = [ploca[r + i][c + i] for i in range(duzina_niza)]
            zgoditak += procjeni(rupa, zeton)
            c +=1
        r+=1

    ##dijagonalu prema gore
    r=0
    while r< Red -3:
        c =0
        while c< Stupac -3:
            rupa = [ploca[r+3- i][c+i] for i in range(duzina_niza)]
            zgoditak+= procjeni(rupa, zeton)
            c+= 1
        r +=1

    return zgoditak

def postavi_na_vrh(ploca, stupac):
    r=0
    while r <Red:
        if ploca[r][stupac] == 0:
            return r
        r +=1


def pobjednicki_potez(ploca, zeton):
    #horizontalo
    c =0
    while c < Stupac - 3:
        r = 0
        while r < Red:
            if ploca[r][c + 1] == zeton and ploca[r][c] == zeton:
                if ploca[r][c + 2] == zeton and ploca[r][c + 3] == zeton:
                    return True
            r += 1
        c += 1

    #vertikalno
    c = 0
    while c < Stupac:
        r = 0
        while r < Red - 3:
            if ploca[r][c]== zeton and ploca[r + 1][c] == zeton:
                if ploca[r+ 2][c] ==zeton and ploca[r +3][c]== zeton:
                    return True
            r += 1
        c += 1

    #diagnoalno gore
    c = 0
    while c <Stupac -3:
        r =0
        while r < Red -3:
            if ploca[r][c]== zeton and ploca[r +1][c +1] == zeton:
                if ploca[r + 2][c + 2] == zeton and ploca[r +3][c +3] == zeton:
                    return True
            r +=1
        c +=1

    # diagonalno dolje
    c =0
    while c < Stupac -3:
        r =3
        while r <Red:
            if ploca[r][c] == zeton and ploca[r - 1][c + 1] == zeton:
                if ploca[r - 2][c + 2] ==zeton and ploca[r -3][c +3] == zeton:
                    return True
            r +=1
        c +=1

    return False


def terminal_node(ploca):
    return pobjednicki_potez(ploca, covjek_zeton) or pobjednicki_potez(ploca, UmjetnaInt_zeton) or len(get_odobrena_lokacija(ploca)) == 0



def polje_postavljanja(ploca, red, stupac, zeton):
    ploca[red][stupac] =zeton

def minimax(ploca, dubina, a, b, igracc):
    
    def zadavanje_vrijednsoti(stupac, new_zgoditak):
        nonlocal vrijednost
        nonlocal stupacumn
        vrijednost =new_zgoditak
        stupacumn =stupac
        
    
    terminal =terminal_node(ploca)
    odobrena_lokacija =get_odobrena_lokacija(ploca)
  
    if terminal or dubina ==0:
        return (
            (None, 1000000000) if pobjednicki_potez(ploca, UmjetnaInt_zeton)
            else (None, -100000000) if pobjednicki_potez(ploca, covjek_zeton)
            else (None, procjeni_poziciju(ploca, UmjetnaInt_zeton)) if dubina == 0
            else (None, 0)
        )

    
    if igracc:
        vrijednost= -math.inf
        stupacumn= random.choice(odobrena_lokacija)
    else:
        vrijednost =math.inf
        stupacumn =random.choice(odobrena_lokacija)
        
    for stupac in odobrena_lokacija:
            
        red= postavi_na_vrh(ploca, stupac)
        B= ploca.copy()
        if igracc:
            polje_postavljanja(B, red, stupac, UmjetnaInt_zeton)
        else:
            polje_postavljanja(B, red, stupac, covjek_zeton)
        if igracc:
            new_zgoditak= minimax(B, dubina - 1, a, b, False)[1]
            if new_zgoditak > vrijednost:
                zadavanje_vrijednsoti(stupac, new_zgoditak)
        else:
            new_zgoditak = minimax(B, dubina - 1, a, b, True)[1]
            if new_zgoditak < vrijednost:
                zadavanje_vrijednsoti(stupac, new_zgoditak)
       
        if igracc:
            a= max(a, vrijednost)
        else: 
            b= min(b, vrijednost)

        if a >= b:
            break
    return stupacumn, vrijednost

def provjera_polja(ploca, stupac):
    return ploca[Red -1][stupac] == 0

def get_odobrena_lokacija(ploca):
    return [stupac for stupac in range(Stupac) if provjera_polja(ploca, stupac)]


def dizajn_ploce(ploca):
    c =0
    while c < Stupac:
        r =0
        while r < Red:
            pygame.draw.rect(sucelje, bijela, (c * dimenzije, r * dimenzije + dimenzije, dimenzije, dimenzije))
            pygame.draw.circle(sucelje, tamno_plava, (c * dimenzije + dimenzije // 2, r * dimenzije + 3 * dimenzije // 2), RADIUS)
            r +=1
        c +=1

    c =0
    
    while c < Stupac:
        r= 0
        while r < Red:
            if ploca[r][c] == covjek_zeton:
                pygame.draw.circle(sucelje, narancasta, (c * dimenzije + dimenzije // 2, height - r * dimenzije - dimenzije // 2), RADIUS)
            elif ploca[r][c] == UmjetnaInt_zeton:
                pygame.draw.circle(sucelje, roza, (c * dimenzije + dimenzije // 2, height - r * dimenzije - dimenzije // 2), RADIUS)
            r +=1
        c +=1

    pygame.display.update()

def postavi_sučelje(dimenzije):
    global RADIUS ,sucelje, myfont
    RADIUS = int(dimenzije / 2-2)
    sucelje = pygame.display.set_mode(size)
    myfont = pygame.font.SysFont("monospace", 45)
    
def polje_igranja():
    ploca =np.zeros((Red, Stupac))
    return ploca


def print_ploca(ploca):
    print(np.flip(ploca, 0))

ploca =polje_igranja()
print_ploca(ploca)
game_over= False

pygame.init()
dimenzije =89

width, height = Stupac *dimenzije, (Red + 1) *dimenzije
size = (width, height)
postavi_sučelje(dimenzije)
covjek =0
UmjetnaInt =1

dizajn_ploce(ploca)
pygame.display.update()

turn = random.randint(covjek, UmjetnaInt)

while not game_over:
    
    def update_game_state():
        global turn
        turn += 1
        turn %= 2
        print_ploca(ploca)
        dizajn_ploce(ploca)

     
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if event.type == pygame.MOUSEMOTION or (turn == covjek and event.type == pygame.MOUSEBUTTONDOWN):
            pygame.draw.rect(sucelje, tamno_zelena, (0, 0, width, dimenzije))
            pygame.display.update()

            if turn == covjek and event.type == pygame.MOUSEBUTTONDOWN:
                posx = event.pos[0]
                stupac = posx // dimenzije

                if provjera_polja(ploca, stupac):
                    red = postavi_na_vrh(ploca, stupac)
                    polje_postavljanja(ploca, red, stupac, covjek_zeton)

                    if pobjednicki_potez(ploca, covjek_zeton):
                        sucelje.blit(myfont.render("Pobjedio s!!!!!!!!", 1, narancasta), (40, 10))
                        game_over = True

                    update_game_state()


    if turn == UmjetnaInt and not game_over:
        stupac, minimax_zgoditak =minimax(ploca, 5, -math.inf, math.inf, True)

        if provjera_polja(ploca, stupac):
            red = postavi_na_vrh(ploca, stupac)
            polje_postavljanja(ploca, red, stupac, UmjetnaInt_zeton)

            if pobjednicki_potez(ploca, UmjetnaInt_zeton):
                sucelje.blit(myfont.render("Pobjedio je AI !!!!!!!!", 1, narancasta), (40, 10))
                game_over =True

            update_game_state()


    if game_over:
        pygame.time.wait(5000)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 1. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 2. 0. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 2. 2. 0. 1. 1.]]
[[0. 0. 0. 0. 0. 